Original code by Dede et al. (https://figshare.com/articles/software/enCas12a_screen_analysis_pipeline/12275642), licensed under CC BY 4.0
https://creativecommons.org/licenses/by/4.0/
Modified by Hamda Ajmal, March 2025

Changes: Extracted relevant portions of the code and applied them to different datasets.

In [ ]:
%matplotlib inline
%pylab inline
import pandas as pd
import scipy.stats as stats
import scipy.cluster.hierarchy as clust
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
reads = pd.read_csv('../InputData/Thompson/raw_counts.txt', sep = "\t")
print(reads.shape)
reads = reads.dropna()  

flucs = reads[reads['GENE'].str.contains('Fluc')]


(83686, 23)


In [5]:
reads[reads['GENE'] == "Non_Tar_RND_control_Non_Tar_RND_control"] # Already removed it so dont need to worry

,gRNA,GENE,A375_D14_R1,A375_D14_R2,A375_D14_R3,A375_D28_R1,A375_D28_R2,A375_D28_R3,MEWO_D14_R1,MEWO_D14_R2,...,MEWO_D28_R3,RPE_D14_R1,RPE_D14_R2,RPE_D14_R3,RPE_D28_R1,RPE_D28_R2,RPE_D28_R3,Control_R1,Control_R2,Control_R3
45959,Non_Tar_RND_control_gRNA100_N_Non_Tar_RND_cont...,Non_Tar_RND_control_Non_Tar_RND_control,1059.0,708.0,1071.0,1261.0,1047.0,1542.0,768.0,789.0,...,905.0,1019.0,1193.0,921.0,410.0,551.0,380.0,802.0,816.0,573.0
45961,Non_Tar_RND_control_gRNA101_N_Non_Tar_RND_cont...,Non_Tar_RND_control_Non_Tar_RND_control,1574.0,1117.0,1439.0,2045.0,1682.0,1718.0,973.0,1216.0,...,1067.0,1666.0,1633.0,1670.0,861.0,717.0,1503.0,818.0,916.0,711.0
45963,Non_Tar_RND_control_gRNA102_N_Non_Tar_RND_cont...,Non_Tar_RND_control_Non_Tar_RND_control,1096.0,1110.0,1255.0,1304.0,1209.0,1463.0,752.0,1039.0,...,886.0,1020.0,1308.0,1460.0,375.0,624.0,885.0,929.0,862.0,580.0
45965,Non_Tar_RND_control_gRNA103_N_Non_Tar_RND_cont...,Non_Tar_RND_control_Non_Tar_RND_control,715.0,942.0,724.0,881.0,1232.0,771.0,755.0,652.0,...,794.0,734.0,1149.0,837.0,218.0,453.0,242.0,528.0,431.0,417.0
45967,Non_Tar_RND_control_gRNA104_N_Non_Tar_RND_cont...,Non_Tar_RND_control_Non_Tar_RND_control,1096.0,1023.0,1074.0,1259.0,1336.0,1700.0,1153.0,943.0,...,1061.0,1206.0,1218.0,1200.0,527.0,706.0,613.0,704.0,671.0,674.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46945,Non_Tar_RND_control_gRNA96_N_Non_Tar_RND_contr...,Non_Tar_RND_control_Non_Tar_RND_control,1472.0,1193.0,1229.0,1345.0,1405.0,1675.0,891.0,840.0,...,938.0,1244.0,1112.0,1935.0,403.0,507.0,893.0,849.0,779.0,644.0
46947,Non_Tar_RND_control_gRNA97_N_Non_Tar_RND_contr...,Non_Tar_RND_control_Non_Tar_RND_control,1393.0,1171.0,1349.0,1732.0,1785.0,1956.0,1178.0,1028.0,...,1216.0,1535.0,1561.0,1345.0,736.0,578.0,639.0,885.0,1031.0,717.0
46949,Non_Tar_RND_control_gRNA98_N_Non_Tar_RND_contr...,Non_Tar_RND_control_Non_Tar_RND_control,636.0,577.0,801.0,844.0,595.0,991.0,682.0,551.0,...,519.0,1004.0,864.0,810.0,437.0,309.0,487.0,521.0,564.0,488.0
46951,Non_Tar_RND_control_gRNA99_N_Non_Tar_RND_contr...,Non_Tar_RND_control_Non_Tar_RND_control,736.0,727.0,664.0,775.0,917.0,914.0,642.0,720.0,...,780.0,1180.0,1326.0,821.0,601.0,446.0,661.0,473.0,625.0,594.0


In [6]:
reads = reads.drop(columns = ['gRNA'])

reads['GENE'] = reads['GENE'].str.strip()
# Also, addition step, take mean of 3 control readings and drop them after
reads['Plasmid'] = reads[['Control_R1', 'Control_R2', 'Control_R3']].mean(axis=1)
#reads.drop(columns = ['Control_R1','Control_R2','Control_R3'])
reads.drop(['Control_R1', 'Control_R2', 'Control_R3'], axis=1, inplace=True)

#reads

,GENE,A375_D14_R1,A375_D14_R2,A375_D14_R3,A375_D28_R1,A375_D28_R2,A375_D28_R3,MEWO_D14_R1,MEWO_D14_R2,MEWO_D14_R3,MEWO_D28_R1,MEWO_D28_R2,MEWO_D28_R3,RPE_D14_R1,RPE_D14_R2,RPE_D14_R3,RPE_D28_R1,RPE_D28_R2,RPE_D28_R3,Plasmid
1,AARS2_AARS,272.0,219.0,298.0,8.0,122.0,50.0,433.0,428.0,489.0,76.0,67.0,118.0,302.0,411.0,260.0,71.0,54.0,43.0,454.000000
3,AARS2_AARS,217.0,273.0,247.0,98.0,82.0,77.0,557.0,436.0,363.0,254.0,83.0,103.0,267.0,181.0,108.0,14.0,6.0,10.0,557.666667
5,AARS2_AARS,216.0,195.0,166.0,74.0,56.0,79.0,589.0,566.0,437.0,100.0,48.0,123.0,112.0,271.0,87.0,4.0,26.0,23.0,700.000000
7,AARS2_AARS,24.0,37.0,84.0,0.0,16.0,26.0,317.0,208.0,236.0,91.0,61.0,89.0,76.0,156.0,23.0,15.0,50.0,0.0,673.333333
9,AARS2_AARS,35.0,16.0,57.0,36.0,24.0,62.0,82.0,97.0,56.0,37.0,2.0,66.0,68.0,106.0,26.0,28.0,26.0,21.0,641.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83677,ZYX_TYMS,840.0,639.0,625.0,149.0,141.0,112.0,1320.0,1154.0,1338.0,887.0,889.0,829.0,727.0,899.0,743.0,293.0,449.0,582.0,1137.666667
83679,ZYX_TYMS,749.0,701.0,704.0,803.0,941.0,634.0,798.0,666.0,700.0,1003.0,626.0,637.0,927.0,1021.0,614.0,350.0,456.0,323.0,613.333333
83681,ZYX_TYMS,547.0,455.0,526.0,128.0,233.0,204.0,833.0,790.0,696.0,877.0,655.0,499.0,600.0,637.0,599.0,167.0,230.0,326.0,690.333333
83683,ZYX_TYMS,949.0,959.0,948.0,1100.0,1436.0,1204.0,979.0,982.0,819.0,1080.0,1149.0,791.0,845.0,936.0,759.0,303.0,202.0,453.0,695.333333


In [7]:
#Non_Tar_RND_control_Non_Tar_RND_control # 498 non-targeting gRNAs need to be removed
reads = reads[reads['GENE'] != 'Non_Tar_RND_control_Non_Tar_RND_control']
numGuides, numSamples = reads.shape

numGuides, numSamples
sample_sum = reads.iloc[:,range(1,numSamples)].sum(0)
sample_sum


(41345, 20)

In [14]:
pseudo=5
reads[ reads.columns[1:]] = reads[ reads.columns[1:]] + pseudo


C:\Users\Hamda\AppData\Local\Temp\ipykernel_107612\2404797452.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reads[ reads.columns[1:]] = reads[ reads.columns[1:]] + pseudo


,GENE,A375_D14_R1,A375_D14_R2,A375_D14_R3,A375_D28_R1,A375_D28_R2,A375_D28_R3,MEWO_D14_R1,MEWO_D14_R2,MEWO_D14_R3,MEWO_D28_R1,MEWO_D28_R2,MEWO_D28_R3,RPE_D14_R1,RPE_D14_R2,RPE_D14_R3,RPE_D28_R1,RPE_D28_R2,RPE_D28_R3,Plasmid
1,AARS2_AARS,277.0,224.0,303.0,13.0,127.0,55.0,438.0,433.0,494.0,81.0,72.0,123.0,307.0,416.0,265.0,76.0,59.0,48.0,459.000000
3,AARS2_AARS,222.0,278.0,252.0,103.0,87.0,82.0,562.0,441.0,368.0,259.0,88.0,108.0,272.0,186.0,113.0,19.0,11.0,15.0,562.666667
5,AARS2_AARS,221.0,200.0,171.0,79.0,61.0,84.0,594.0,571.0,442.0,105.0,53.0,128.0,117.0,276.0,92.0,9.0,31.0,28.0,705.000000
7,AARS2_AARS,29.0,42.0,89.0,5.0,21.0,31.0,322.0,213.0,241.0,96.0,66.0,94.0,81.0,161.0,28.0,20.0,55.0,5.0,678.333333
9,AARS2_AARS,40.0,21.0,62.0,41.0,29.0,67.0,87.0,102.0,61.0,42.0,7.0,71.0,73.0,111.0,31.0,33.0,31.0,26.0,646.000000


In [15]:
meanReads = reads.iloc[:,range(1,numSamples)].mean(0) # colmean
meanReads

A375_D14_R1     980.287314
A375_D14_R2     923.700810
A375_D14_R3     969.409143
A375_D28_R1    1006.149353
A375_D28_R2    1097.982586
A375_D28_R3     955.224259
MEWO_D14_R1     997.273915
MEWO_D14_R2     988.931793
MEWO_D14_R3     958.698126
MEWO_D28_R1    1021.286298
MEWO_D28_R2     924.964397
MEWO_D28_R3     950.155762
RPE_D14_R1      995.655775
RPE_D14_R2     1068.498174
RPE_D14_R3      922.025783
RPE_D28_R1     1291.674664
RPE_D28_R2      965.839521
RPE_D28_R3      984.512952
Plasmid         911.711791
dtype: float64

In [16]:
numGuides, numSamples = reads.shape

numGuides, numSamples 

(41345, 20)

In [19]:
normed = pd.DataFrame(index=reads.index, columns=reads.columns) # empty data frame
normed['GENE'] = reads.iloc[:, 0]
#reads.shape
#pseudo has already been added to reads
#The numpy.tile() function constructs a new array by repeating array – ‘arr’, 
#the number of times we want to repeat as per repetitions. 
#The resulted array will have dimensions max(arr.ndim, repetitions) where, 
#repetitions is the length of repetitions. If arr.ndim > repetitions,
#reps is promoted to arr.ndim by pre-pending 1’s to it.
#If arr.ndim < repetitions, reps is promoted to arr.ndim by pre-pending new axis. Syntax : 
normed[ normed.columns[1:] ] =   (reads.iloc[:, range(1, numSamples)] ) / np.tile(meanReads.values, [numGuides, 1]) * 500  # normalize to mean 500 read
normed.shape
#why 500? anyway...

(41345, 20)

In [30]:
fc = pd.DataFrame(index=reads.index, columns=reads.columns[:-1]) # non-normalised
fc['GENE'] = reads.iloc[:, 0]  # first column is gene name
#fc.shape # share is for dimensions - nice
numFCsamples = fc.shape[1]-1   # number of columns for which to calculate FC
numFCsamples
fc.shape

pseudo = 0 # remmeber pseudocount is already in the data 

# calculation fc wrt to last column, fc of each cell line/replicate wrt to sample at t0
fc[ fc.columns[1:] ] =log2( (normed[ normed.columns[1:-1]] + pseudo) / np.tile( normed[ normed.columns[-1]] + pseudo , [numFCsamples, 1]).T )
fc.head()


,GENE,A375_D14_R1,A375_D14_R2,A375_D14_R3,A375_D28_R1,A375_D28_R2,A375_D28_R3,MEWO_D14_R1,MEWO_D14_R2,MEWO_D14_R3,MEWO_D28_R1,MEWO_D28_R2,MEWO_D28_R3,RPE_D14_R1,RPE_D14_R2,RPE_D14_R3,RPE_D28_R1,RPE_D28_R2,RPE_D28_R3
1,AARS2_AARS,-0.833235,-1.053843,-0.687704,-5.284105,-2.121871,-3.128252,-0.196975,-0.201420,0.033518,-2.666238,-2.693245,-1.959422,-0.707325,-0.370845,-0.808731,-3.097016,-3.042913,-3.368220
3,AARS2_AARS,-1.446348,-1.036043,-1.247385,-2.591831,-2.961399,-2.845846,-0.131122,-0.468795,-0.685074,-1.283066,-2.697525,-2.440835,-1.175743,-1.825913,-2.332187,-5.390802,-5.759911,-5.340078
5,AARS2_AARS,-1.778204,-1.836471,-2.132155,-3.299893,-3.798948,-3.136424,-0.376572,-0.421426,-0.746076,-2.910972,-3.754379,-2.521066,-2.718184,-1.581890,-2.954147,-6.794147,-4.590489,-4.764957
7,AARS2_AARS,-4.652496,-4.032381,-3.018645,-7.226117,-5.281739,-4.518916,-1.204346,-1.788434,-1.565460,-2.984626,-3.382276,-2.910848,-3.193070,-2.303868,-4.614725,-5.586515,-3.707697,-7.194755
9,AARS2_AARS,-4.118089,-4.961921,-3.469722,-4.120033,-4.745615,-3.336563,-3.021859,-2.780258,-3.477152,-4.106811,-6.548855,-3.245229,-3.272635,-2.769909,-4.397423,-4.793589,-4.464400,-4.745783


In [31]:
fc[['GENE1', 'GENE2']] = fc['GENE'].str.split('_', expand=True)
fc

,GENE,A375_D14_R1,A375_D14_R2,A375_D14_R3,A375_D28_R1,A375_D28_R2,A375_D28_R3,MEWO_D14_R1,MEWO_D14_R2,MEWO_D14_R3,...,MEWO_D28_R2,MEWO_D28_R3,RPE_D14_R1,RPE_D14_R2,RPE_D14_R3,RPE_D28_R1,RPE_D28_R2,RPE_D28_R3,GENE1,GENE2
1,AARS2_AARS,-0.833235,-1.053843,-0.687704,-5.284105,-2.121871,-3.128252,-0.196975,-0.201420,0.033518,...,-2.693245,-1.959422,-0.707325,-0.370845,-0.808731,-3.097016,-3.042913,-3.368220,AARS2,AARS
3,AARS2_AARS,-1.446348,-1.036043,-1.247385,-2.591831,-2.961399,-2.845846,-0.131122,-0.468795,-0.685074,...,-2.697525,-2.440835,-1.175743,-1.825913,-2.332187,-5.390802,-5.759911,-5.340078,AARS2,AARS
5,AARS2_AARS,-1.778204,-1.836471,-2.132155,-3.299893,-3.798948,-3.136424,-0.376572,-0.421426,-0.746076,...,-3.754379,-2.521066,-2.718184,-1.581890,-2.954147,-6.794147,-4.590489,-4.764957,AARS2,AARS
7,AARS2_AARS,-4.652496,-4.032381,-3.018645,-7.226117,-5.281739,-4.518916,-1.204346,-1.788434,-1.565460,...,-3.382276,-2.910848,-3.193070,-2.303868,-4.614725,-5.586515,-3.707697,-7.194755,AARS2,AARS
9,AARS2_AARS,-4.118089,-4.961921,-3.469722,-4.120033,-4.745615,-3.336563,-3.021859,-2.780258,-3.477152,...,-6.548855,-3.245229,-3.272635,-2.769909,-4.397423,-4.793589,-4.464400,-4.745783,AARS2,AARS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83677,ZYX_TYMS,-0.540008,-0.846120,-0.947509,-3.033597,-3.236570,-3.355086,0.084176,-0.096817,0.160556,...,-0.374878,-0.513872,-0.769558,-0.566945,-0.627524,-2.441613,-1.414846,-1.071805,ZYX,TYMS
83679,ZYX_TYMS,0.181553,0.172436,0.108873,0.243776,0.345250,-0.019830,0.247603,0.000635,0.116740,...,0.008435,-0.005398,0.464876,0.501639,-0.014675,-1.303159,-0.506824,-1.025521,ZYX,TYMS
83681,ZYX_TYMS,-0.437663,-0.614919,-0.477521,-2.528473,-1.814949,-1.801463,0.139834,0.075957,-0.060789,...,-0.096059,-0.523867,-0.327839,-0.344066,-0.219385,-2.517889,-1.648250,-1.181706,ZYX,TYMS
83683,ZYX_TYMS,0.341321,0.442144,0.355907,0.515738,0.772751,0.720439,0.361205,0.377715,0.162104,...,0.699710,0.125140,0.152352,0.197218,0.109302,-1.687704,-1.841617,-0.723527,ZYX,TYMS


In [32]:
cells = list(['A375_D14','RPE_D14','MEWO_D14','A375_D28','RPE_D28','MEWO_D28' ])
cols = list(['GENE']) + cells
fc_merge = pd.DataFrame( columns=cols, index=fc.index, dtype=float)
fc_merge.GENE = fc.GENE
#print(fc_merge)
for cell in cells:
    samples = [x for x in fc.columns if cell in x]
    fc_merge[cell] = fc[ samples ].mean(1)
fc_merge['GENE1'] = fc.GENE1
fc_merge['GENE2'] = fc.GENE2
fc_merge.head()

,GENE,A375_D14,RPE_D14,MEWO_D14,A375_D28,RPE_D28,MEWO_D28,GENE1,GENE2
1,AARS2_AARS,-0.858261,-0.628967,-0.121626,-3.511410,-3.169383,-2.439635,AARS2,AARS
3,AARS2_AARS,-1.243259,-1.777948,-0.428330,-2.799692,-5.496930,-2.140475,AARS2,AARS
5,AARS2_AARS,-1.915610,-2.418073,-0.514691,-3.411755,-5.383198,-3.062139,AARS2,AARS
7,AARS2_AARS,-3.901174,-3.370554,-1.519413,-5.675591,-5.496322,-3.092583,AARS2,AARS
9,AARS2_AARS,-4.183244,-3.479989,-3.093090,-4.067404,-4.667924,-4.633632,AARS2,AARS


In [34]:
# Get SMF for each gene (gene-CTRL pairs)
is_ctrl = where( (fc_merge.GENE1=='Fluc') | (fc_merge.GENE2=='Fluc') )[0]
len(is_ctrl) # FLuc is only on one side

is_ctrl1 = where( fc_merge.GENE1=='Fluc' )[0]
is_ctrl2 = where( fc_merge.GENE2=='Fluc' )[0]



In [35]:
smf_gene1 = fc_merge.iloc[is_ctrl2].groupby('GENE1').mean(numeric_only = True) # calculate means of multiple runs of same gene
smf_gene2 = fc_merge.iloc[is_ctrl1].groupby('GENE2').mean(numeric_only = True)


       A375_D14   RPE_D14  MEWO_D14  A375_D28   RPE_D28  MEWO_D28
GENE1                                                            
AARS  -1.358687 -0.990985 -0.744243 -1.822747 -2.718671 -1.382895
AARS2 -0.004042  0.211690  0.131120 -0.973168 -1.444982 -0.300751
AASS   0.312078  0.327489  0.035748  0.486592 -0.628792  0.130878
ABCB6  0.166490  0.029154  0.093892  0.264667 -0.980032  0.158597
ABCB7  0.402252  0.516114  0.180449  0.338673 -0.855920  0.225500
Empty DataFrame
Columns: [A375_D14, RPE_D14, MEWO_D14, A375_D28, RPE_D28, MEWO_D28]
Index: []


In [37]:
smf_guide1 = fc_merge.iloc[is_ctrl2].groupby('GENE1')
smf_guide1
smf_guide2 = fc_merge.iloc[is_ctrl1].groupby('GENE2')


In [38]:
smf_gene = smf_gene1.join(smf_gene2, lsuffix='_Aposn', rsuffix='_Bposn')
df_filtered = smf_gene.filter(like='_Bposn', axis=1)
smf_gene = smf_gene.drop(columns=df_filtered.columns)#print(smf_gene)

smf_gene
#Now funny thing is that in this dataset, control is alsways on Aposn.
#Therefore, we need to drop the _Bposn columns

,A375_D14_Aposn,RPE_D14_Aposn,MEWO_D14_Aposn,A375_D28_Aposn,RPE_D28_Aposn,MEWO_D28_Aposn
GENE1,,,,,,
AARS,-1.358687,-0.990985,-0.744243,-1.822747,-2.718671,-1.382895
AARS2,-0.004042,0.211690,0.131120,-0.973168,-1.444982,-0.300751
AASS,0.312078,0.327489,0.035748,0.486592,-0.628792,0.130878
ABCB6,0.166490,0.029154,0.093892,0.264667,-0.980032,0.158597
ABCB7,0.402252,0.516114,0.180449,0.338673,-0.855920,0.225500
...,...,...,...,...,...,...
ZP2,0.352252,0.607783,0.145171,0.365682,-0.444805,0.276788
ZP4,0.296238,0.387544,0.179773,0.286469,-0.761297,0.298775
ZSWIM2,0.194384,0.167883,-0.002339,0.391007,-0.955203,0.131374


In [39]:
# whats happening here? A_posn and B_posn of each gene is averaged
# Perhaps not needed for this dataset but go with the flow 
for cell in cells:
    smf_gene[cell] = smf_gene[ [cell + '_Aposn']].mean(1)
    smf_gene.drop( [cell + '_Aposn'], axis=1, inplace=True )
    
smf_gene

,A375_D14,RPE_D14,MEWO_D14,A375_D28,RPE_D28,MEWO_D28
GENE1,,,,,,
AARS,-1.358687,-0.990985,-0.744243,-1.822747,-2.718671,-1.382895
AARS2,-0.004042,0.211690,0.131120,-0.973168,-1.444982,-0.300751
AASS,0.312078,0.327489,0.035748,0.486592,-0.628792,0.130878
ABCB6,0.166490,0.029154,0.093892,0.264667,-0.980032,0.158597
ABCB7,0.402252,0.516114,0.180449,0.338673,-0.855920,0.225500
...,...,...,...,...,...,...
ZP2,0.352252,0.607783,0.145171,0.365682,-0.444805,0.276788
ZP4,0.296238,0.387544,0.179773,0.286469,-0.761297,0.298775
ZSWIM2,0.194384,0.167883,-0.002339,0.391007,-0.955203,0.131374


In [40]:
pairs = fc_merge[(fc_merge['GENE1'] != "Fluc") & (fc_merge['GENE2'] != "Fluc")] # These are all experiments without control,
pairs =  pairs[["GENE1", "GENE2"]]
pairs =pairs.drop_duplicates(keep='first')
pairs.insert(2, "GENE1_GENE2",np.tile("ZZ",len(pairs)), True)



pairs.columns
for ind in pairs.index:
    g1 = pairs.loc[ind, 'GENE1']
    g2 = pairs.loc[ind, 'GENE2']
    newval = g1 + "_" + g2
    if g1 > g2:
        newval = g2 + "_" + g1
    pairs.loc[ind,"GENE1_GENE2"] = newval

pairs.drop_duplicates(subset="GENE1_GENE2", keep="first",inplace = True)


1191
1191


,GENE1,GENE2,GENE1_GENE2
1,AARS2,AARS,AARS_AARS2
91,ABCB7,ABCB6,ABCB6_ABCB7
161,ABCF1,ABCF3,ABCF1_ABCF3
241,ABHD12B,ABHD12,ABHD12_ABHD12B
311,ABL1,EGFR,ABL1_EGFR
...,...,...,...
83417,ZSWIM3,CENPF,CENPF_ZSWIM3
83429,ZSWIM3,WDHD1,WDHD1_ZSWIM3
83527,ZYX,EXO1,EXO1_ZYX
83539,ZYX,MKI67,MKI67_ZYX


In [41]:
dLFC = pd.DataFrame( index=list(pairs.GENE1 + "_" + pairs.GENE2), columns=fc_merge.columns[:-2], dtype=float)
dLFC.head(3)
dLFC.drop('GENE', inplace = True, axis = 1)
dLFC

,A375_D14,RPE_D14,MEWO_D14,A375_D28,RPE_D28,MEWO_D28
AARS2_AARS,NaN,NaN,NaN,NaN,NaN,NaN
ABCB7_ABCB6,NaN,NaN,NaN,NaN,NaN,NaN
ABCF1_ABCF3,NaN,NaN,NaN,NaN,NaN,NaN
ABHD12B_ABHD12,NaN,NaN,NaN,NaN,NaN,NaN
ABL1_EGFR,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
ZSWIM3_CENPF,NaN,NaN,NaN,NaN,NaN,NaN
ZSWIM3_WDHD1,NaN,NaN,NaN,NaN,NaN,NaN
ZYX_EXO1,NaN,NaN,NaN,NaN,NaN,NaN
ZYX_MKI67,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
fc = fc_merge
for pair_idx in pairs.index:
    g1 = pairs.loc[pair_idx].GENE1
    g2 = pairs.loc[pair_idx].GENE2
    expt_idx  = list( where( ( (fc.GENE1==g1) & (fc.GENE2==g2) ) | ( (fc.GENE1==g2) & (fc.GENE2==g1) )  )[0] )
    if ( len(expt_idx)==0 ):
        continue
    smf_sum = smf_gene.loc[g1] + smf_gene.loc[g2]
    expt = fc.iloc[ expt_idx ]
    genepair = g1 + "_" + g2
    dLFC.loc[genepair] = expt.median(0, numeric_only=True) - smf_sum

In [43]:
zdLFC = pd.DataFrame( index=dLFC.index, columns=dLFC.columns, dtype=float ) 
#To compare across screens, we converted dLFC scores to a Z score, zdLFC, by truncating
#the top and bottom 2.5% of dLFC scores (Additional file 1: Fig. S2J, Additional
#file 3: Table S2, Additional file 4: Table S3). At a zdLFC score < − 3, all three
#screens showed high concordance, with 19 of 24 (79%) synthetic lethals present in at
#least two out of three cell lines and 14 of 24 (58%) present in all three (Fig. 4a, b)

#Z-transformation of distribution of dLFC (zdLFC) after truncating top/bottom 2.5%
#of values approximates a normal distribution.

percentile = 2.5
for col in zdLFC.columns:
    #print(col)
    top = np.percentile( dLFC.loc[:,col], percentile)
    bot = np.percentile( dLFC.loc[:,col], 100-percentile)
    #print(top,bot)
    mu = dLFC.iloc[ where( (dLFC[col]>top) & (dLFC[col]<bot))[0] ][col].mean() # This is because our dist is truncated normal
    std = dLFC.iloc[ where( (dLFC[col]>top) & (dLFC[col]<bot))[0] ][col].std()
    #print(mu, std)
    zdLFC[col] = (dLFC[col] - mu) / std

In [47]:
def reindex_alphbetically(df):
    result = []
    for index, row in df.iterrows():
        a, b = index.split('_')
        if a < b:
            result.append(f'{a}_{b}')
        else:
            result.append(f'{b}_{a}')
    
    
    return(result)


zdLFC.index = reindex_alphbetically(zdLFC)
zdLFC.to_csv("zdLFC Output/Thompson_zdLFC.csv", index=True)  # Set index=False to exclude the index column
